# AI TICKET CLASSIFIER - COMPLETE ASSESMENT SOLUTION

This script implements a complete ticket classification system with:
1. Data setup and analysis
2. NLP pipeline with text processing
3. Machine Learning model training and evaluation
4. Named Entity Recognition(NER)

# Importing Everything we need

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk.tree import Tree
import re

# Download required NLTK data

In [2]:
try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

try:
  nltk.data.find('tokenizers/punkt_tab')
except LookupError:
  nltk.download('punkt_tab')

try:
  nltk.data.find('corpora/stopwords')
except LookupError:
  nltk.download('stopwords')

try:
  nltk.data.find('corpora/wordnet')
except LookupError:
  nltk.download('wordnet')

try:
  nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
  nltk.download('averaged_perceptron_tagger')

try:
  nltk.data.find('chunkers/maxent_ne_chunker')
except LookupError:
  nltk.download('maxent_ne_chunker')

try:
  nltk.data.find('corpora/words')
except LookupError:
  nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


# Creating Ticket Classifier Class

In [12]:
class TicketClassifier:
  def __init__(self):   # javascript we use this and in python we use self
     self.vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
     self.classifier = MultinomialNB()
     self.lemmatizer = WordNetLemmatizer()
     self.stop_words = set(stopwords.words('english'))

  def load_and_analyze_data(self, csv_file=None):
    """TASK 1: Load an analyze the ticket data"""
    print("=== TASK 1: AUTOMATE TICKET CLASSIFICATION ===")
    print("Objective: Set up the dataset \n")

    if csv_file:
      try:
        #Load actual CSV file
        self.df = pd.read_csv(csv_file)
        print(f"Loaded {len(self.df)} tickets from {csv_file}")

        #Add Data validation
        #check for missing values in ticket_text
        if self.df['ticket_text'].isna().any():
          print("Found missing ticket_text values, filling with default text")
          self.df['ticket_text'] = self.df['ticket_text'].fillna('No description provided')

        #ensure all ticket_text entries are strings
        self.df['ticket_text'] = self.df['ticket_text'].astype(str)

        #fill any empty categories with empty string
        self.df['category'] = self.df['category'].fillna('')


      except FileNotFoundError:
        print("Could not find csv file exception")
    else:
      print("Could not find csv file")


    print("\n Data Collection Complete:")
    print(f" Total ticket loaded: {len(self.df)}")

    print("\n 2. Quick check - Category Distribution:")
    category_counts = self.df['category'].value_counts()
    print(f" Technical: {category_counts.get('Technical', 0)} tickets")
    print(f" Billing: {category_counts.get('Billing', 0)} tickets")
    print(f" General: {category_counts.get('General', 0)} tickets")

    # Count unlabeled tickets
    unlabeled_count = len(self.df[self.df['category'] == ''])
    labeled_count = len(self.df[self.df['category'] != ''])

    print(f"\n Unlabeled tickets: {unlabeled_count}")
    print(f" Labeled tickets: {labeled_count}")

    #Separate labeled and unlabeled data
    self.labeled_df = self.df[self.df['category'] != ''].copy()
    self.unlabeled_df = self.df[self.df['category'] == ''].copy()

    print("\n Data Split:")
    print(f" Labeled tickets: {len(self.labeled_df)}")
    print(f" Unlabeled tickets: {len(self.unlabeled_df)}\n")

    #Split labeled data: 10 training, 3 validation, 2 test

    train_data, temp_data = train_test_split(self.labeled_df, test_size=5, random_state=42, stratify=self.labeled_df['category'])
    val_data, test_data = train_test_split(temp_data, test_size=2, random_state=42)

    self.train_data = train_data
    self.val_data = val_data
    self.test_data = test_data

    print(f" Training set: {len(self.train_data)} tickets") #same thing in javascript as objt.length
    print(f" Validation set: {len(self.val_data)} tickets")
    print(f" Test set: {len(self.test_data)} tickets")

    return self.df

  def extract_named_entities(self, text):
     """ Extract named entities using NLTK's NER"""
     try:
       tokens = word_tokenize(text)
       pos_tag = pos_tag(tokens)
       tree = ne_chunk(pos_tag)

       entities = []
       for chunk in tree:
         if isinstance(chunk, Tree):
           entities.append(' '.join([token for token, pos in chunk.leaves()]))
       return entities

     except:
       return []

  def nlp_pipeline(self, text):
    """  Task 2: Complete NBLP Pipeline for text processing"""
    # Add a validation for empty/null text
    if pd.isna(text) or not isinstance(text, str) or len(text.strip()) == 0:
      return {
          'processed_text': 'empty ticket',
          'tokens': ['empty', 'ticket'],
          'key_terms': ['empty', ' ticket'],
          'entities': []
      }

    # 1. Convert to lowercase and remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())

    # 2. Tokenization
    tokens = word_tokenize(text)

    # 3. Remove stop words
    tokens = [token for token in tokens if token not in self.stop_words and len(token) > 2]

    # 4. Lemmatization
    lemmatized_tokens = [self.lemmatizer.lemmatize(token) for token in tokens]


    # handle any empty result
    if not lemmatized_tokens:
      lemmatized_tokens = ['short', 'text']

    # 5. Extract Key terms using simple frequency and NER
    processed_text = ' '.join(lemmatized_tokens)

    # ensure processed_text is not empty
    if not processed_text.strip():
      processed_text = 'short text'

    # extract named entities
    try:
      entities = self.extract_named_entities(text)

    except:
      entities = []

    # Combine processed text with entities
    key_terms = lemmatized_tokens[:5] + entities # top 5 key terms + entities

    return {
        'processed_text': processed_text,
        'tokens': lemmatized_tokens,
        'key_terms': key_terms[:3] if key_terms else ['short', 'text'],
        'entities': entities
    }

  def build_nlp_pipeline(self):
    """Task 2: Build NLP pipeline and process  all tickets"""
    print("\n === Task 2: NLP Pipeline == \n")
    print("Text Cleanup Steps:")
    print("1. Remove stop words (eg, 'the', 'this')")
    print("2. Lemmatize words (eg 'crashed' -> 'crash')")
    print("3. Tokenize text into words")
    print("4. Extract named entities and key terms \n")

    #process all tickets
    processed_tickets = []
    print("Processing tickets with NLP pipeline")
    for idx, row in self.df.iterrows():
      processed = self.nlp_pipeline(row['ticket_text'])
      processed_tickets.append({
          'ticket_id': row['ticket_id'],
          'original_text': row['ticket_text'],
          'processed_text': processed['processed_text'],
          'key_terms': processed['key_terms'],
          'entities': processed['entities'],
          'category': row['category']

      })

      if idx == 0:
        print(f"Example Ticket {row['ticket_id']}")
        print(f"Original {row['ticket_text']}")
        print(f"Key Terms {processed['key_terms']}")
        print(f"Entities {processed['entities']}")

    self.processed_data = pd.DataFrame(processed_tickets)
    print(f"\n NLP pipeline applied to all {len(self.df)} tickets")

    return self.processed_data

  def train_and_evaluate_model(self):
    """ Task 3: Train and evaluate the ML model"""
    print("\n == Task 3: Train and evaluate ML Model")
    print("Objective: Train and test the model \n")

    # 1. Algorithm Selection
    print("1. Algorithm Selection:")
    print("Primary: Naive Bayes - fast and effective for text classification")
    print("Secondary: Logistic Regression - good baseline for comparison")
    print("Clustering: K-Means for unlabeled ticket analysis \n")

    #Prepare training data
    train_texts = self.train_data['ticket_text'].tolist()
    train_labels = self.train_data['category'].tolist()

    # 2. Train the model
    print("2. Model Training")

    #processed training text through NLP pipeline
    processed_train_texts = []
    for text in train_texts:
      processed = self.nlp_pipeline(text)
      processed_train_texts.append(processed['processed_text'])

    #Add validation: check for empty processed texts
    if any(not text.strip() for text in processed_train_texts):
      print(" Warning: some tickets produced empty text after processing")
      processed_train_texts = [text if text.strip() else 'empty ticket' for text in processed_train_texts]

    # Vectorize the text (basically doing [1, 2] -> [[1], [2]])
    X_train = self.vectorizer.fit_transform(processed_train_texts)
    y_train = train_labels

    # Train Naive Bayes
    self.classifier.fit(X_train, y_train)
    print(f"  Naive Bayes trained on {len(train_texts)} tickets")

    # Train Logistic Regression for comparison
    self.lr_classifier = LogisticRegression(random_state=42)
    self.lr_classifier.fit(X_train, y_train)
    print(f" Logistic Regression trained on {len(train_texts)} tickets")

    # 3. Validation Testing
    print("\n 3. Validation Testing:")
    val_texts = self.val_data['ticket_text'].tolist()
    val_labels = self.val_data['category'].tolist()

    processed_val_texts = []
    for text in val_texts:
      processed = self.nlp_pipeline(text)
      processed_val_texts.append(processed['processed_text'])

    X_val = self.vectorizer.transform(processed_val_texts)
    val_predictions = self.classifier.predict(X_val)
    val_accuracy = accuracy_score(val_labels, val_predictions)

    print(f" Validation Accuracy: {val_accuracy:.2%}")
    print(f" Validation Results: {sum(val_predictions == val_labels)}/{len(val_labels)} correct")

    # 4. Test Set Evaluation
    print("\n 4. Test Set Evaluation")
    test_texts = self.test_data['ticket_text'].tolist()
    test_labels = self.test_data['category'].tolist()

    processed_test_texts = []
    for text in test_texts:
      processed = self.nlp_pipeline(text)
      processed_test_texts.append(processed['processed_text'])

    X_test = self.vectorizer.transform(processed_test_texts)
    test_predictions = self.classifier.predict(X_test)
    test_accuracy = accuracy_score(test_labels, test_predictions)

    print(f" Test Accuracy: {test_accuracy:.2%}")
    print(f" Test Results: {sum(test_predictions == test_labels)}/{len(test_labels)} correct")




















# Invoking our class

In [14]:
#Initialize Ticket Classifier
classifier = TicketClassifier()

# Task 1: Load and analyze data
classifier.load_and_analyze_data(csv_file='/content/tickets.csv')

# Task 2: Build NLP Pipeline (remember nlp_pipeline is being called inside this function)
classifier.build_nlp_pipeline()

# Task 3: Train and evaluate model
results = classifier.train_and_evaluate_model()


=== TASK 1: AUTOMATE TICKET CLASSIFICATION ===
Objective: Set up the dataset 

Loaded 20 tickets from /content/tickets.csv

 Data Collection Complete:
 Total ticket loaded: 20

 2. Quick check - Category Distribution:
 Technical: 5 tickets
 Billing: 5 tickets
 General: 5 tickets

 Unlabeled tickets: 5
 Labeled tickets: 15

 Data Split:
 Labeled tickets: 15
 Unlabeled tickets: 5

 Training set: 10 tickets
 Validation set: 3 tickets
 Test set: 2 tickets

 === Task 2: NLP Pipeline == 

Text Cleanup Steps:
1. Remove stop words (eg, 'the', 'this')
2. Lemmatize words (eg 'crashed' -> 'crash')
3. Tokenize text into words
4. Extract named entities and key terms 

Processing tickets with NLP pipeline
Example Ticket 1
Original App crashes whenever I try to open it.
Key Terms ['app', 'crash', 'whenever']
Entities []

 NLP pipeline applied to all 20 tickets

 == Task 3: Train and evaluate ML Model
Objective: Train and test the model 

1. Algorithm Selection:
Primary: Naive Bayes - fast and effecti